In [44]:
import requests
import defs
import pandas as pd

In [45]:
session =  requests.Session()
ins_df = pd.read_pickle("instruments.pk1")


In [46]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [47]:
ins_df.shape

(68, 5)

In [48]:
ins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         68 non-null     object
 1   type         68 non-null     object
 2   displayName  68 non-null     object
 3   pipLocation  68 non-null     int64 
 4   marginRate   68 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.8+ KB


In [49]:
ins_df.columns

Index(['name', 'type', 'displayName', 'pipLocation', 'marginRate'], dtype='object')

In [50]:
ins_df.name.unique()

array(['HKD_JPY', 'USD_SGD', 'EUR_SEK', 'GBP_PLN', 'EUR_PLN', 'GBP_NZD',
       'EUR_NOK', 'USD_CZK', 'EUR_GBP', 'CHF_HKD', 'EUR_CZK', 'NZD_JPY',
       'EUR_HUF', 'CAD_HKD', 'USD_CHF', 'AUD_HKD', 'NZD_CHF', 'AUD_CHF',
       'GBP_CHF', 'USD_THB', 'GBP_CAD', 'EUR_HKD', 'CHF_JPY', 'GBP_HKD',
       'EUR_NZD', 'AUD_SGD', 'EUR_JPY', 'EUR_TRY', 'USD_JPY', 'SGD_JPY',
       'GBP_ZAR', 'ZAR_JPY', 'AUD_JPY', 'SGD_CHF', 'EUR_CHF', 'NZD_CAD',
       'USD_CNH', 'USD_TRY', 'GBP_JPY', 'EUR_SGD', 'AUD_USD', 'USD_NOK',
       'NZD_SGD', 'USD_CAD', 'EUR_AUD', 'TRY_JPY', 'CAD_JPY', 'USD_ZAR',
       'USD_DKK', 'EUR_ZAR', 'USD_PLN', 'GBP_AUD', 'NZD_USD', 'USD_HKD',
       'CHF_ZAR', 'EUR_USD', 'GBP_SGD', 'USD_SEK', 'USD_HUF', 'EUR_CAD',
       'EUR_DKK', 'AUD_CAD', 'USD_MXN', 'GBP_USD', 'CAD_CHF', 'AUD_NZD',
       'CAD_SGD', 'NZD_HKD'], dtype=object)

In [51]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )

    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [52]:
code, res = fetch_candles("EUR_USD", 10, "H1")

In [53]:
code

200

In [54]:
res

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 4204,
   'time': '2024-07-29T15:00:00.000000000Z',
   'bid': {'o': '1.08148', 'h': '1.08232', 'l': '1.08147', 'c': '1.08181'},
   'mid': {'o': '1.08156', 'h': '1.08239', 'l': '1.08154', 'c': '1.08189'},
   'ask': {'o': '1.08163', 'h': '1.08246', 'l': '1.08162', 'c': '1.08197'}},
  {'complete': True,
   'volume': 3435,
   'time': '2024-07-29T16:00:00.000000000Z',
   'bid': {'o': '1.08184', 'h': '1.08230', 'l': '1.08142', 'c': '1.08207'},
   'mid': {'o': '1.08192', 'h': '1.08238', 'l': '1.08150', 'c': '1.08215'},
   'ask': {'o': '1.08199', 'h': '1.08246', 'l': '1.08158', 'c': '1.08223'}},
  {'complete': True,
   'volume': 3187,
   'time': '2024-07-29T17:00:00.000000000Z',
   'bid': {'o': '1.08209', 'h': '1.08253', 'l': '1.08188', 'c': '1.08242'},
   'mid': {'o': '1.08216', 'h': '1.08260', 'l': '1.08196', 'c': '1.08250'},
   'ask': {'o': '1.08224', 'h': '1.08268', 'l': '1.08203', 'c': '1.08258'}},

In [55]:
def get_candle_df(json_response):

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue

        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
    
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)



In [56]:
code, res = fetch_candles("EUR_USD", 10, "H1")

In [57]:
df = get_candle_df(res)

In [58]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-07-29T15:00:00.000000000Z,4204,1.08156,1.08239,1.08154,1.08189,1.08148,1.08232,1.08147,1.08181,1.08163,1.08246,1.08162,1.08197
1,2024-07-29T16:00:00.000000000Z,3435,1.08192,1.08238,1.08150,1.08215,1.08184,1.08230,1.08142,1.08207,1.08199,1.08246,1.08158,1.08223
2,2024-07-29T17:00:00.000000000Z,3187,1.08216,1.08260,1.08196,1.08250,1.08209,1.08253,1.08188,1.08242,1.08224,1.08268,1.08203,1.08258
3,2024-07-29T18:00:00.000000000Z,3019,1.08248,1.08250,1.08194,1.08216,1.08240,1.08243,1.08186,1.08209,1.08256,1.08258,1.08202,1.08224
4,2024-07-29T19:00:00.000000000Z,3087,1.08218,1.08257,1.08218,1.08237,1.08210,1.08249,1.08210,1.08229,1.08226,1.08265,1.08225,1.08245


In [59]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"his_data/{pair}_{granularity}.pk1")

In [60]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candle_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [61]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")

EUR_USD loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2023-12-06T10:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
USD_JPY loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
USD_CHF loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
USD_CAD loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
GBP_USD loaded 3999 candles from 2023-12-06T08:00:00.000000000Z to 2024-07-29T23:00:00.000000000Z
GBP_JPY loaded 3999 